In [13]:
from astroquery.simbad import Simbad
import pandas as pd
from time import sleep

def printMessier():
    """
    Retrieves details for all 110 Messier objects using astroquery's Simbad interface.
    Returns a pandas DataFrame with the collected information.
    """
    # Configure Simbad to return additional fields
    custom_simbad = Simbad()
    votable_fields = Simbad.list_votable_fields()
    for field in votable_fields:
       print(field['name'  ] , "\t" , field['description'] )
 
printMessier()

mesDiameter 	 Collection of stellar diameters.
mesPM 	 Collection of proper motions.
mesISO 	 Infrared Space Observatory (ISO) observing log.
mesSpT 	 Collection of spectral types.
allfluxes 	 all flux/magnitudes U,B,V,I,J,H,K,u_,g_,r_,i_,z_
ident 	 Identifiers of an astronomical object
flux 	 Magnitude/Flux information about an astronomical object
mesOtype 	 Other object types associated with an object with origins
mesPLX 	 Collection of trigonometric parallaxes.
otypedef 	 all names and definitions for the object types
mesDistance 	 Collection of distances (pc, kpc or Mpc) by several means.
otypes 	 List of all object types associated with an object
mesVar 	 Collection of stellar variability types and periods.
mesXmm 	 XMM observing log.
mesVelocities 	 Collection of HRV, Vlsr, cz and redshifts.
has_ref 	 Associations between astronomical objects and their bibliographic references
mesRot 	 Stellar Rotational Velocities.
biblio 	 Bibliography
ids 	 all names concatenated with pipe
mes

In [21]:
from astroquery.simbad import Simbad
import pandas as pd
from time import sleep

def printMessier():
    """
    Retrieves details for all 110 Messier objects using astroquery's Simbad interface.
    Returns a pandas DataFrame with the collected information.
    """
    # Configure Simbad to return additional fields
    custom_simbad = Simbad()
    votable_fields = Simbad.list_votable_fields()

    #custom_simbad.add_votable_fields('ident','basic','velocity',)
    all_fields = votable_fields['name'].tolist()

# Add all fields to the custom Simbad instance
    custom_simbad.add_votable_fields('V')
    
    # Initialize lists to store results
    results = []
    
    # Loop through all Messier objects (M1 to M110)
    for i in range(1, 2):
        messier_id = f"M {i}"
        try:
            # Query Simbad database
            result_table = custom_simbad.query_object(messier_id)
            
            if result_table is not None:
                # Extract data from the result
                print(result_table)
                for result in result_table:
                    for key in result.keys():
                        print(f"{key}: {result[key]}")
                objectIds = Simbad.query_objectids(messier_id)
                print(objectIds)    
                data = {
                    'Messier': messier_id,
                    'Name': result_table['MAIN_ID'][0].decode(),
                    'RA': result_table['RA'][0],
                    'DEC': result_table['DEC'][0],
                    'Type': result_table['OTYPE'][0].decode() if result_table['OTYPE'][0] is not None else 'Unknown',
                    'Morphology': result_table['MORPHTYPE'][0].decode() if result_table['MORPHTYPE'][0] is not None else 'Unknown',
                    'Dimensions': result_table['DIM'][0].decode() if result_table['DIM'][0] is not None else 'Unknown',
                    'Distance': result_table['Distance_distance'][0] if result_table['Distance_distance'][0] is not None else None,
                    'Magnitude': result_table['MAG'][0] if result_table['MAG'][0] is not None else None
                }
                results.append(data)
                print(f"Successfully retrieved data for {messier_id}")
            else:
                print(f"No data found for {messier_id}")
                
            # Add a small delay to avoid overwhelming the server
            sleep(1)
            
        except Exception as e:
            print(f"Error retrieving data for {messier_id}: {str(e)}")
            continue
    
    # Convert results to DataFrame
    df = pd.DataFrame(results)
    
    # Clean up the data
    if 'Distance' in df.columns:
        df['Distance'] = pd.to_numeric(df['Distance'], errors='coerce')
    if 'Magnitude' in df.columns:
        df['Magnitude'] = pd.to_numeric(df['Magnitude'], errors='coerce')
    
    return df

def save_results(df, csv_path='messier_objects.csv'):
    """
    Saves the DataFrame to a CSV file.
    
    Parameters:
    df (pandas.DataFrame): DataFrame containing Messier object data
    csv_path (str): Path where the CSV file should be saved
    """
    df.to_csv(csv_path, index=False)
    print(f"Results saved to {csv_path}")

def main():
    print("Starting Messier object data collection...")
    df = printMessier()
    
    # Display basic statistics
    print("\nData collection complete!")
    print(f"Total objects retrieved: {len(df)}")
    
    # Save the results
    save_results(df)
    
    # Display some example data
    print("\nSample of collected data:")
    print(df.head())
    
    # Display some basic statistics
    print("\nBasic statistics for numerical values:")
    print(df[['Distance', 'Magnitude']].describe())

 
printMessier()

main_id  ra dec coo_err_maj ... coo_wavelength coo_bibcode  V  matched_id
        deg deg     mas     ...                                          
------- --- --- ----------- ... -------------- ----------- --- ----------
           id          
-----------------------
PCCS2 030 G184.54-05.78
  SRGA J053431.8+220103
         3HWC J0534+220
         2HWC J0534+220
         eHWC J0534+220
  1RXS J053431.2+220218
        2AGL J0534+2205
        SNR G184.6-05.8
            1H 0531+219
            1M 0531+219
                    ...
     SWIFT J0534.6+2204
     SWIFT J0534.5+2200
     GRS G184.60 -05.80
                    W 9
    NVSS J053428+220202
        ARGO J0535+2203
     3FGL J0534.5+2201i
          TeV J0534+220
      3FHL J0534.5+2201
     4FGL J0534.5+2201i
Length = 64 rows
Error retrieving data for M 1: 'MAIN_ID'


""
